*Note: this notebook was generated by dsflow*

# create table meteo_agg

## \[1- init and config\] refresh tables and set output path

In [1]:
import os
import json
import datetime as dt
from pyspark.sql import SparkSession
from dsflow_core.helpers import DsflowContext

spark = SparkSession.builder.getOrCreate()
dsflow = DsflowContext.create()

spark

In [2]:
dsflow.load_tables()

INFO - meteoparis was successful imported/updated


,database,tableName,isTemporary
0,default,meteoparis,False


In [3]:
default_task_specs = """
    {"sink_path": "/data/tables/meteo_agg/ds=2017-11-24",
     "ds": "2017-11-24"}
    """

In [4]:
task_specs_raw = os.environ.get('TASK_SPECS', default_task_specs)
task_specs = json.loads(task_specs_raw)

# dsflow alerts if something looks wrong
dsflow.validade_task_specs(task_specs)
print(task_specs)

{'sink_path': '/data/tables/meteo_agg/ds=2017-11-24', 'ds': '2017-11-24'}


## \[2- process data\] process data based on SQL

In [7]:
df = spark.sql("""

  SELECT cast(forecast as timestamp) as forecast_hour,
     sum(total_water_precipitation) as total_precipitation
  FROM meteoparis
  WHERE ds = '2017-11-24'
  GROUP BY 1
  ORDER BY 1

""")

Execute query and show first 10 rows

In [8]:
dsflow.display(df)

Showing 10 random rows


,forecast_hour,total_precipitation
0,2017-11-23 18:00:00,NaN
1,2017-11-23 19:00:00,5.963379
2,2017-11-23 20:00:00,14.315918
3,2017-11-23 21:00:00,15.423340
4,2017-11-23 22:00:00,601.946777
5,2017-11-23 23:00:00,1391.389648
6,2017-11-24 00:00:00,2159.660156
7,2017-11-24 01:00:00,2448.616211
8,2017-11-24 02:00:00,2621.346680
9,2017-11-24 03:00:00,2718.778809


HINT: Register dataframe as a temporary view

In [9]:
df.registerTempTable("tmp_meteo_agg")

In [10]:
other_df = spark.sql("""

  SELECT COUNT(*)
  FROM tmp_meteo_agg

""")

dsflow.display(other_df)

Showing 10 random rows


,count(1)
0,37


In [11]:
# print schema
df.printSchema()

root
 |-- forecast_hour: timestamp (nullable = true)
 |-- total_precipitation: double (nullable = true)



## \[3- write on disk\] write the output as parquet

In [12]:
final_dataset = df

In [13]:
(final_dataset
 .write
 .mode("overwrite")
 .parquet(task_specs["sink_path"])
)

In [ ]:
dsflow.validade_task_output(task_specs)